# OcampoATAC2

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.OcampoATAC2)

class OcampoATAC2(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        """
        Normalize a PyTorch tensor of counts to TPM (Transcripts Per Million) then
        transforms with log1p.
        """
        lengths = torch.tensor(
            self.preprocess_dependencies[0], device=x.device, dtype=x.dtype
        )

        # Normalize by length
        tpm = 1000 * (x / lengths.unsqueeze(0))

        # Scale to TPM (Transcripts Per Million)
        tpm = 1e6 * (tpm / tpm.sum(dim=1, keepdim=True))

        # Apply log1p transformation
        tpm_log1p = torch.log1p(tpm)

        return tpm_log1p[:, self.preprocess_dependencies[1]]

    def postprocess(self, x):
        return x



In [3]:
model = pya.models.OcampoATAC2()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'ocampoatac2'
model.metadata["data_type"] = 'atac'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2023
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Morandini, Francesco, et al. \"ATAC-clock: An aging clock based on chromatin accessibility.\" GeroScience (2023): 1-18."
model.metadata["doi"] = "https://doi.org/10.1007/s11357-023-00986-0"
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download directly with curl

In [5]:
supplementary_url = "https://static-content.springer.com/esm/art%3A10.1007%2Fs11357-023-00986-0/MediaObjects/11357_2023_986_MOESM9_ESM.tsv"
supplementary_file_name = "peaks.tsv"
os.system(f"curl -o {supplementary_file_name} {supplementary_url}")

0

#### Download GitHub repository

In [6]:
github_url = "https://github.com/SunScript0/ATAC-clock.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

## Load features

#### From CSV file

In [7]:
df = pd.read_table('ATAC-clock/clocks/parallel/2023-03-19_10-06_tpm_all_samples/final_coefs.tsv', index_col=0)
df['feature'] = df.index.tolist()
df['coefficient'] = df['coef']
df = df[df.coef != 0]
peaks = pd.read_table('peaks.tsv')

model.features = peaks['desc'].tolist()
base_model_features = df['feature'][0:-1].tolist()
model.base_model_features = peaks.loc[base_model_features, 'desc'].tolist()

## Load weights into base model

In [8]:
weights = torch.tensor(df['coefficient'][0:-1].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][-1]])

<ipython-input-8-1490e47125f3>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept = torch.tensor([df['coefficient'][-1]])


#### Linear model

In [9]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [10]:
model.reference_values = None

## Load preprocess and postprocess objects

In [11]:
def parse_genomic_location(loc: str):
    chrom, positions = loc.split(':')
    start, end = map(int, positions.split('-'))
    length = end - start
    return length
    
model.preprocess_name = 'tpm_norm_log1p'
model.preprocess_dependencies = [[parse_genomic_location(peak) for peak in peaks['desc'].tolist()]]

indices = [model.features.index(item) for item in model.base_model_features]
model.preprocess_dependencies += [indices]

In [12]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [13]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Morandini, Francesco, et al. "ATAC-clock: An aging clock based '
             'on chromatin accessibility." GeroScience (2023): 1-18.',
 'clock_name': 'ocampoatac2',
 'data_type': 'atac',
 'doi': 'https://doi.org/10.1007/s11357-023-00986-0',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2023}
reference_values: None
preprocess_name: 'tpm_norm_log1p'
preprocess_dependencies: [[591,
  1449,
  1113,
  866,
  1200,
  700,
  698,
  1399,
  436,
  751,
  586,
  2517,
  2654,
  1086,
  969,
  1301,
  890,
  783,
  2284,
  482,
  1236,
  1837,
  1716,
  1294,
  1218,
  860,
  1716,
  1505,
  1175,
  1225,
  1248,
  1022,
  2021,
  985,
  1708,
  646,
  750,
  779,
  1258,
  552,
  1516,
  509,
  1255,
  1590,
  1094,
  1568,
  1338,
  933,
  1527,
  562,
  769,
  528,
  63

## Basic test

In [14]:
torch.manual_seed(42)
input = torch.abs(torch.randn(10, len(model.features), dtype=float))
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[33.3062],
        [32.6990],
        [28.1495],
        [29.3579],
        [35.7597],
        [38.2028],
        [27.6359],
        [31.4678],
        [31.1109],
        [34.6479]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [15]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [16]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: peaks.tsv
Deleted folder: ATAC-clock
